In [1]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' }

from scrapy.selector import Selector

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#1.) Market Cap greater than $2B & 3-month trading volume of 200,000

#2.) Revenue growth exceeding 3-4% for the last 2 years

#3.) Earnings growth in the last 2 years 

#4.) Positive earnings surprise sometime within the past 2 quarters (GC: Might not be able to find)

#5.) ROE and/or ROIC growing in the last 3 years

#6.) PEG ratio is less than the industry average

#7.) Current Ratio greater than 1

#8.) P/E less than the industry average 

#9.) Operating Margin exceeding sector average, preferably a company within a profitable industry.

In [ ]:
"""#This method gets tickers from a CSV instead of the Charles Schwab website. It is more reliable, but requires more user input.
#Identically, it calls in more tickers than the computer can handle.
# Load the CSV file
file_path = "Holdings_details_Consumer_Discretionary_ETF.csv"
data = pd.read_csv(file_path)

# Extract the tickers into a Python list
tickers = data['TICKER'].tolist()

# Print the list to confirm
print(tickers)"""

In [11]:
#All of the values pulled from Yahoo are strings. This function converts them to numbers.
def convert_values(value):
    """
    Convert a string with T, B, M, %, or plain numbers into their respective number values.
    """
    
    if ',' in value:
        return float(value.replace(',', ''))
    elif 'T' in value:
        return float(value.replace('T', '')) * 1_000_000_000_000
    elif 'B' in value:
        return float(value.replace('B', '')) * 1_000_000_000
    elif 'M' in value:
        return float(value.replace('M', '')) * 1_000_000
    elif '%' in value:
        return float(value.replace('%', ''))
    else:
        return float(value)

In [13]:
#Dataframe with math done in the function

tickers = ['AMZN','MCD']

def get_data(Ticker):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36' } 
    #                                                                 profile
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/profile/', headers=headers).text
    response = Selector(text=html)
    Sector = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[2]/section[2]/div/dl/div[1]/dd/a/text()').extract_first()
    #
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/profile/', headers=headers).text
    response = Selector(text=html)
    Industry = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[2]/section[2]/div/dl/div[2]/a/text()').extract_first()
    #                                                                 key-statistics
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/key-statistics/', headers=headers).text
    response = Selector(text=html)
    Market_Cap = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/table[1]/tbody/tr[1]/td[2]/text()').extract_first()
    Market_Cap = convert_values(Market_Cap)
    #
    _3_Month_Trading_Volume = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/div/section[2]/div/section[2]/table/tbody/tr[1]/td[2]/text()').extract_first()
    _3_Month_Trading_Volume = convert_values(_3_Month_Trading_Volume)
    #
    PEG_Ratio = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/table[1]/tbody/tr[5]/td[2]/text()').extract_first()
    PEG_Ratio = convert_values(PEG_Ratio)
    #
    Current_Ratio = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]/div/section[5]/table/tbody/tr[5]/td[2]/text()').extract_first()
    Current_Ratio = convert_values(Current_Ratio)
    #
    Trailing_P_E_Ratio = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[2]/div[2]/table[1]/tbody/tr[3]/td[2]/text()').extract_first()
    Trailing_P_E_Ratio = convert_values(Trailing_P_E_Ratio)
    #
    Operating_Margin = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/div/section[1]/div/section[2]/table/tbody/tr[2]/td[2]/text()').extract_first()
    Operating_Margin = convert_values(Operating_Margin)
    #                                                                 financials
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/financials/', headers=headers).text
    response = Selector(text=html)
    #Don't include Revenue in Dataframe. We are using it to create Revenue Growth over the last two years.
    Revenue_Most_Recent_Annual_Statement = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[1]/div[3]/text()').extract_first()
    Revenue_Most_Recent_Annual_Statement = convert_values(Revenue_Most_Recent_Annual_Statement)
    #
    Revenue_One_Annual_Statement_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[1]/div[4]/text()').extract_first()
    Revenue_One_Annual_Statement_Ago = convert_values(Revenue_One_Annual_Statement_Ago)
    #
    Revenue_Two_Annual_Statements_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[1]/div[5]/text()').extract_first()
    Revenue_Two_Annual_Statements_Ago = convert_values(Revenue_Two_Annual_Statements_Ago)
#
    Revenue_Growth_Last_Year_Percent = ((Revenue_Most_Recent_Annual_Statement - Revenue_One_Annual_Statement_Ago) / Revenue_One_Annual_Statement_Ago)*100
    Revenue_Growth_One_Year_Ago_Percent = ((Revenue_One_Annual_Statement_Ago - Revenue_Two_Annual_Statements_Ago) / Revenue_Two_Annual_Statements_Ago)*100
#
    #Don't include Earnings in Dataframe. We are using it to create Revenue Growth over the last two years.
    #
    Earnings_Most_Recent_Annual_Statement = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[11]/div[3]/text()').extract_first()
    Earnings_Most_Recent_Annual_Statement = convert_values(Earnings_Most_Recent_Annual_Statement)
    #
    Earnings_One_Annual_Statement_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[11]/div[4]/text()').extract_first()
    Earnings_One_Annual_Statement_Ago = convert_values(Earnings_One_Annual_Statement_Ago)
    #
    Earnings_Two_Annual_Statements_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[11]/div[5]/text()').extract_first()
    Earnings_Two_Annual_Statements_Ago = convert_values(Earnings_Two_Annual_Statements_Ago)
#
    Earnings_Growth_Last_Year_Percent = ((Earnings_Most_Recent_Annual_Statement - Earnings_One_Annual_Statement_Ago) / Earnings_One_Annual_Statement_Ago)*100
    Earnings_Growth_One_Year_Ago_Percent = ((Earnings_One_Annual_Statement_Ago - Earnings_Two_Annual_Statements_Ago) / Earnings_Two_Annual_Statements_Ago)*100
#
    #                                                                  analysis
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/analysis/', headers=headers).text
    response = Selector(text=html)
    Earnings_Surprise_Most_Recent_Quarter_Percent = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[4]/div/table/tbody/tr[4]/td[5]/text()').extract_first()
    Earnings_Surprise_Most_Recent_Quarter_Percent = convert_values(Earnings_Surprise_Most_Recent_Quarter_Percent)
    #
    Earnings_Surprise_One_Quarter_Ago_Percent = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/section[4]/div/table/tbody/tr[4]/td[4]/text()').extract_first()
    Earnings_Surprise_One_Quarter_Ago_Percent = convert_values(Earnings_Surprise_One_Quarter_Ago_Percent)
    #                                                                 balance-sheet
    html = requests.get(f'https://finance.yahoo.com/quote/{Ticker}/balance-sheet/', headers=headers).text
    response = Selector(text=html)
    #Don't include Shareholders equity
    Shareholders_Equity_Most_Recent_Annual_Statement = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[3]/div[2]/text()').extract_first()
    Shareholders_Equity_Most_Recent_Annual_Statement = convert_values(Shareholders_Equity_Most_Recent_Annual_Statement)
    #
    Shareholders_Equity_One_Annual_Statement_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[3]/div[3]/text()').extract_first()
    Shareholders_Equity_One_Annual_Statement_Ago = convert_values(Shareholders_Equity_One_Annual_Statement_Ago)
    #
    Shareholders_Equity_Two_Annual_Statements_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[3]/div[4]/text()').extract_first()
    Shareholders_Equity_Two_Annual_Statements_Ago = convert_values(Shareholders_Equity_Two_Annual_Statements_Ago)
    #
    Shareholders_Equity_Three_Annual_Statements_Ago = response.xpath('//*[@id="nimbus-app"]/section/section/section/article/article/section/div/div[1]/div[2]/div[3]/div[5]/text()').extract_first()
    Shareholders_Equity_Three_Annual_Statements_Ago = convert_values(Shareholders_Equity_Three_Annual_Statements_Ago)
#Return on Equity (ROE) = Net Income ÷ Average Shareholders’ Equity
#Average Shareholders’ Equity = ( Last Statement - Previous Statement ) / 2
#
    Return_on_Equity_Most_Recent_Year = Earnings_Most_Recent_Annual_Statement / ((Shareholders_Equity_Most_Recent_Annual_Statement + Shareholders_Equity_One_Annual_Statement_Ago)/2)
    Return_on_Equity_One_Year_Ago = Earnings_One_Annual_Statement_Ago / ((Shareholders_Equity_One_Annual_Statement_Ago + Shareholders_Equity_Two_Annual_Statements_Ago)/2)
    Return_on_Equity_Two_Years_Ago = Earnings_Two_Annual_Statements_Ago / ((Shareholders_Equity_Two_Annual_Statements_Ago + Shareholders_Equity_Three_Annual_Statements_Ago)/2)
#
    #
    return Market_Cap, _3_Month_Trading_Volume, PEG_Ratio, Current_Ratio, Trailing_P_E_Ratio, Operating_Margin, \
Earnings_Surprise_Most_Recent_Quarter_Percent, Earnings_Surprise_One_Quarter_Ago_Percent, \
Revenue_Growth_Last_Year_Percent, Revenue_Growth_One_Year_Ago_Percent,\
Earnings_Growth_Last_Year_Percent, Earnings_Growth_One_Year_Ago_Percent, Return_on_Equity_Most_Recent_Year, Return_on_Equity_One_Year_Ago, \
Return_on_Equity_Two_Years_Ago, Sector, Industry

# Initalize a new pandas DataFrame
df = pd.DataFrame(columns = ['Ticker','Market_Cap','_3_Month_Trading_Volume','PEG_Ratio','Current_Ratio','Trailing_P_E_Ratio','Operating_Margin',
                             'Earnings_Surprise_Most_Recent_Quarter_Percent','Earnings_Surprise_One_Quarter_Ago_Percent',
                             'Revenue_Growth_Last_Year_Percent', 'Revenue_Growth_One_Year_Ago_Percent', 'Earnings_Growth_Last_Year_Percent', 
                            'Earnings_Growth_One_Year_Ago_Percent','Return_on_Equity_Most_Recent_Year','Return_on_Equity_One_Year_Ago', 
                            'Return_on_Equity_Two_Years_Ago','Sector','Industry'])

# Iterate through list of tickers and save values to our df DataFrame
for Ticker in tickers:
    Market_Cap, _3_Month_Trading_Volume, PEG_Ratio, Current_Ratio, Trailing_P_E_Ratio, Operating_Margin, \
    Earnings_Surprise_Most_Recent_Quarter_Percent, Earnings_Surprise_One_Quarter_Ago_Percent, \
    Revenue_Growth_Last_Year_Percent, Revenue_Growth_One_Year_Ago_Percent, \
    Earnings_Growth_Last_Year_Percent, Earnings_Growth_One_Year_Ago_Percent, Return_on_Equity_Most_Recent_Year, Return_on_Equity_One_Year_Ago, \
    Return_on_Equity_Two_Years_Ago, Sector, Industry = get_data(Ticker)
    
    df = pd.concat([df, pd.DataFrame({'Ticker':[Ticker], 'Market_Cap':[Market_Cap], '_3_Month_Trading_Volume':[_3_Month_Trading_Volume],
                                      'PEG_Ratio':[PEG_Ratio], 'Current_Ratio':[Current_Ratio], 'Trailing_P_E_Ratio':[Trailing_P_E_Ratio],
                                      'Operating_Margin':[Operating_Margin], 
                                      'Earnings_Surprise_Most_Recent_Quarter_Percent':[Earnings_Surprise_Most_Recent_Quarter_Percent], 
                                      'Earnings_Surprise_One_Quarter_Ago_Percent':[Earnings_Surprise_One_Quarter_Ago_Percent],
                                      'Revenue_Growth_Last_Year_Percent':[Revenue_Growth_Last_Year_Percent], 
                                      'Revenue_Growth_One_Year_Ago_Percent':[Revenue_Growth_One_Year_Ago_Percent],
                                      'Earnings_Growth_Last_Year_Percent':[Earnings_Growth_Last_Year_Percent], 
                                      'Earnings_Growth_One_Year_Ago_Percent':[Earnings_Growth_One_Year_Ago_Percent],
                                      'Return_on_Equity_Most_Recent_Year':[Return_on_Equity_Most_Recent_Year],
                                      'Return_on_Equity_One_Year_Ago':[Return_on_Equity_One_Year_Ago], 
                                      'Return_on_Equity_Two_Years_Ago':[Return_on_Equity_Two_Years_Ago], 'Sector':[Sector], 
                                      'Industry':[Industry]})], ignore_index=True)

# Print the df DataFrame
df.head()

C:\Users\Win10Admin\AppData\Local\Temp\ipykernel_10408\3745507246.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'Ticker':[Ticker], 'Market_Cap':[Market_Cap], '_3_Month_Trading_Volume':[_3_Month_Trading_Volume],


,Ticker,Market_Cap,_3_Month_Trading_Volume,PEG_Ratio,Current_Ratio,Trailing_P_E_Ratio,Operating_Margin,Earnings_Surprise_Most_Recent_Quarter_Percent,Earnings_Surprise_One_Quarter_Ago_Percent,Revenue_Growth_Last_Year_Percent,Revenue_Growth_One_Year_Ago_Percent,Earnings_Growth_Last_Year_Percent,Earnings_Growth_One_Year_Ago_Percent,Return_on_Equity_Most_Recent_Year,Return_on_Equity_One_Year_Ago,Return_on_Equity_Two_Years_Ago,Sector,Industry
0,AMZN,2.330000e+12,37710000.0,1.81,1.09,47.29,10.96,25.17,22.92,11.829574,9.399517,-1217.744306,-108.158494,0.174898,-0.019150,0.288056,Consumer Cyclical,Internet Retail
1,MCD,2.075300e+11,3490000.0,2.61,0.78,25.43,46.97,0.88,-3.18,9.969115,-0.173536,37.093275,-18.128081,-1.581460,-1.165064,-1.214431,Consumer Cyclical,Restaurants
